# Analisi delle analogie tramite similarità nei vettori di incorporamento

## *"re - uomo + donna = ..."*

In [1]:
import gensim.downloader as api # Gensim è una libreria open source per la modellazione di argomenti 
                                # senza supervisione, l'indicizzazione di documenti, il recupero per 
                                # somiglianza e altre funzionalità di elaborazione del linguaggio 
                                # naturale
                                # (la useremo per caricare un modello preaddestrato di Embeddings)
                                # 'pip install gensim' per installarla
import numpy as np

### Word2Vec

Word2vec è una semplice rete neurale artificiale a due strati progettata per elaborare il linguaggio naturale.    
Richiede in ingresso un corpus di testo e restituisce un insieme di vettori che rappresentano la distribuzione semantica delle parole nel testo.    
Per ogni parola contenuta nel corpus, in modo univoco, viene costruito un vettore in modo da rappresentarla come un punto nello spazio multidimensionale creato (spazio latente del modello). In questo spazio, le parole saranno più vicine se riconosciute come semanticamente più simili.    

In [2]:
# caricamento modello
def load_word2vec_model():
    print("Caricamento del modello Word2Vec...")
    return api.load('word2vec-google-news-300')  # modello pre-addestrata su una parte del dataset 
                                                 # di Google News 
                                                 # (circa 100 miliardi di parole). Gestisce uno 
                                                 # spazio latente a 300 dimensioni

model = load_word2vec_model()

Caricamento del modello Word2Vec...


In [3]:
# funzione per la ricerca di parole analoghe/simili
def find_analogy(model, word1, word2, word3):
    try:
        result = model.most_similar(positive=[word1, word3], negative=[word2], topn=1)
        return result[0][0]
    except KeyError:
        return None

In [4]:
find_analogy(model, "king", "man", "woman")

'queen'

In [5]:
# funzione per il calcolo della similarità del coseno
def cosine_similarity(vec1, vec2):
    return 1 - (np.dot(vec1, vec2.T) / ((np.linalg.norm(vec1) * np.linalg.norm(vec2.T)) + 1e-8))  # n.b. abbiamo aggiunto "1 - ..." all'inizio per avere una misura di "errore": 
                                                                                                  # avremo così numeri tra 0 e 1 che ci indicheranno quanto sono differenti i due vettori

In [6]:
# analisi differenti modalità per il calcolo delle analogie
def print_analogy_explanation(model, word1, word2, word3, result):
    print(f"\nAnalogia: {word1} - {word2} + {word3} = ???    ... {result}")

    vec_relation = model[word1] - model[word2] + model[word3]

    parola_simile = model.similar_by_vector(vec_relation, topn=1)[0][0]
    similarity = cosine_similarity(vec_relation, model[result])

    print(f"\nLa parola più simile, calcolata tramite {word1} - {word2} + {word3} è: {parola_simile}")
    print(f"\nLa parola più simile, calcolata tramite `model.most_similar` è: {result}")
    print(f"\nLa similarità del coseno tra il vettore di '{result}' (model.most_similar) e di '{parola_simile}' ({word3} - {word1} + {word2}) è: {similarity:.4f}\n\n")

In [7]:
# esempio classico di re-regina-uomo-donna
result = find_analogy(model, "king", "man", "woman")
print_analogy_explanation(model, "king", "man", "woman", result)


Analogia: king - man + woman = ???    ... queen

La parola più simile, calcolata tramite king - man + woman è: king

La parola più simile, calcolata tramite `model.most_similar` è: queen

La similarità del coseno tra il vettore di 'queen' (model.most_similar) e di 'king' (woman - king + man) è: 0.2699




In [8]:
# esplorazione ulteriori analogie
def explore_analogies(model):
    print("\nEsploriamo alcune analogie interessanti:")
    analogies = [
        ("king", "man", "woman"),
        ("paris", "france", "italy"),
        ("doctor", "man", "woman"),
        ("highway", "car", "train"),
        ("boy", "man", "woman")
    ]
    
    for word1, word2, word3 in analogies:
        result = find_analogy(model, word1, word2, word3)
        if result:
            print_analogy_explanation(model, word1, word2, word3, result)
        else:
            print(f"\nNon è stato possibile trovare un'analogia per: {word1} : {word2} :: {word3} : ?\n\n")

explore_analogies(model)


Esploriamo alcune analogie interessanti:

Analogia: king - man + woman = ???    ... queen

La parola più simile, calcolata tramite king - man + woman è: king

La parola più simile, calcolata tramite `model.most_similar` è: queen

La similarità del coseno tra il vettore di 'queen' (model.most_similar) e di 'king' (woman - king + man) è: 0.2699



Analogia: paris - france + italy = ???    ... lohan

La parola più simile, calcolata tramite paris - france + italy è: paris

La parola più simile, calcolata tramite `model.most_similar` è: lohan

La similarità del coseno tra il vettore di 'lohan' (model.most_similar) e di 'paris' (italy - paris + france) è: 0.4971



Analogia: doctor - man + woman = ???    ... gynecologist

La parola più simile, calcolata tramite doctor - man + woman è: doctor

La parola più simile, calcolata tramite `model.most_similar` è: gynecologist

La similarità del coseno tra il vettore di 'gynecologist' (model.most_similar) e di 'doctor' (woman - doctor + man) è: 0.27

In [9]:
# ...try it yourself !
#
# Provate a creare le vostre analogie e verificatele con il modello
# Analizzate i risultati e discutete perché alcune analogie funzionano meglio di altre
# Riflettete su possibili bias o limitazioni del modello emersi durante l'esplorazione